In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
ROWS_PER_FRAME = 543  # number of landmarks per frame

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

In [7]:
test_path = "1363575346.parquet"

In [4]:
train = pd.read_csv('train.csv')

# Add ordinally Encoded Sign (assign number to each sign name)
train['sign_ord'] = train['sign'].astype('category').cat.codes

# Dictionaries to translate sign <-> ordinal encoded sign
SIGN2ORD = train[['sign', 'sign_ord']].set_index('sign').squeeze().to_dict()
ORD2SIGN = train[['sign_ord', 'sign']].set_index('sign_ord').squeeze().to_dict()

In [5]:
def predict_sign(path):
    demo_raw_data = load_relevant_data_subset(path)
    interpreter = tf.lite.Interpreter("model.tflite")
    found_signatures = list(interpreter.get_signature_list().keys())
    prediction_fn = interpreter.get_signature_runner("serving_default")

    output = prediction_fn(inputs=demo_raw_data)
    sign = output['outputs'].argmax()

    print("PRED : ", ORD2SIGN.get(sign), f'[{sign}]')
    df = train[train['path'].str.contains(test_path)]
    print("TRUE : ", df.sign.values[0], f'[{df.sign_ord.values[0]}]')
    return ORD2SIGN.get(sign)


In [8]:
predict_sign(test_path)

PRED :  pretty [178]
TRUE :  pretty [178]


'pretty'